In [ ]:
import pandas as pd

#.............................Load The dense Matrix.................................

dft= pd.read_csv('dense_matrix.csv')


dft.loc[dft['Rating'] ==1, 'Rating'] = 0
dft.loc[dft['Rating'] ==2, 'Rating'] = 0
dft.loc[dft['Rating'] ==3, 'Rating'] = 1
dft.loc[dft['Rating'] ==4, 'Rating'] = 2
dft.loc[dft['Rating'] ==5, 'Rating'] = 2

dftt= dft.astype(int)
docs=[]
for index, row in dftt.iterrows():
    present_words_with_count = []
    for i, value in enumerate(row):
        for x in range(value):
            present_words_with_count.append(i)
    docs.append(present_words_with_count)

from tqdm import tqdm
import numpy as np

#............................Obtain sentiment TOPIC DISTRIBUTION OF THE REVIEWS WITH LDA...............................................................................

def distributiont1(t):
    D = len(docs)  # number of documents
    V = dftt.shape[1]  # size of the vocabulary
    T = t  # number of topics

    alpha = 1 / T  # the parameter of the Dirichlet prior on the per-document topic distributions
    beta = 1 / T  # the parameter of the Dirichlet prior on the per-topic word distribution

    z_d_n = [[0 for _ in range(len(d))] for d in docs]  # z_i_j
    theta_d_z = np.zeros((D, T))
    phi_z_w = np.zeros((T, V))
    n_d = np.zeros((D))
    n_z = np.zeros((T))

    ## Initialize the parameters
    # m: doc id
    for d, doc in enumerate(docs):
        # n: id of word inside document, w: id of the word globally
        for n, w in enumerate(doc):
            # assign a topic randomly to words
            z_d_n[d][n] = n % T
            # get the topic for word n in document d
            z = z_d_n[d][n]
            # keep track of our counts
            theta_d_z[d][z] += 1
            phi_z_w[z, w] += 1
            n_z[z] += 1
            n_d[d] += 1

    for iteration in tqdm(range(10)):
        for d, doc in enumerate(docs):
            for n, w in enumerate(doc):
                # get the topic for word n in document m
                z = z_d_n[d][n]

                # decrement counts for word w with associated topic z
                theta_d_z[d][z] -= 1
                phi_z_w[z, w] -= 1
                n_z[z] -= 1

                # sample new topic from a multinomial according to our formula
                p_d_t = (theta_d_z[d] + alpha) / (n_d[d] - 1 + T * alpha)
                p_t_w = (phi_z_w[:, w] + beta) / (n_z + V * beta)
                p_z = p_d_t * p_t_w
                p_z /= np.sum(p_z)
                new_z = np.random.multinomial(1, p_z).argmax()

                # set z as the new topic and increment counts
                z_d_n[d][n] = new_z
                theta_d_z[d][new_z] += 1
                phi_z_w[new_z, w] += 1
                n_z[new_z] += 1
    dfc = pd.DataFrame()
    for i in range(D):
        AR = pd.Series(theta_d_z[i] / sum(theta_d_z[i]))
        dfc = pd.concat([dfc, pd.DataFrame([AR])], ignore_index=True)
    dfc['sentiment'] = dft['Rating'].astype(int)
    dfc = dfc.dropna()
    X=NN_Classi(Linera_Discriminant_Analysis(dfc)[0],len(Linera_Discriminant_Analysis(dfc)[1]))

    return X

#........................Obtain Class Specific Features from LDIS..............................................

def Linera_Discriminant_Analysis(final_matrix):
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
    lda1= LDA()
    x_vals= final_matrix.drop(['sentiment'], axis='columns')
    y_vals=final_matrix.overall_rating
    LDA_transformed= lda1.fit_transform(x_vals,y_vals)
    topic_encoded_reviews= pd.DataFrame(LDA_transformed)
    topic_encoded_reviews['sentiment']=dft['Rating']
    topic_encoded_reviews.update(topic_encoded_reviews.select_dtypes(include=[np.number]).abs())# since NN doesnt get negative values.
    
    return topic_encoded_reviews,lda1.explained_variance_ratio_

#.......................Classification with MLP.................................................................................


def NN_Classi(final_matrixx,ins):
    import tensorflow as tf
    from tensorflow import keras
    from keras import models
    from keras import layers
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.losses import SparseCategoricalCrossentropy
    from sklearn.model_selection import train_test_split
    from tensorflow.keras import regularizers
    from sklearn.metrics import classification_report, confusion_matrix, precision_score

    model = keras.Sequential([keras.layers.Dense(32, input_shape=(ins,), activation='relu'),
                              keras.layers.Dense(3, activation='softmax')
                              ])
    x_vals = final_matrixx.drop(['sentiment'], axis='columns')
    y_vals = final_matrixx.sentiment
    
    #..................................Random sampling to get test results from random splits..............................
    for i in range(20):

        X_train_val, X_test, y_train_val, y_test = train_test_split(x_vals, y_vals, test_size=0.2, random_state=i,stratify=y)

        # Step 2: Split train + validation into train and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=i,stratify=y)

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=20)
     
        y_pred = model.predict(X_test, verbose=2)

    
        predictions = np.argmax(y_pred, axis=1)
        report = classification_report(y_test, predictions)
        print(report)
        
        print(model.evaluate(X_test, y_test))


    return model.evaluate(X_test, y_test)


distributiont1(768)# This is the requested topics for semantic representation.......